In [94]:
import numpy as np
import scipy
import common

import matplotlib.pyplot as plt
import seaborn as sns

from ConfigParser import SafeConfigParser

parser = SafeConfigParser()
parser.read("../config.ini")

width = int(parser.get("Sampling", "width"))
height = int(parser.get("Sampling", "height"))

amp_min = float(parser.get("Sampling", "amp_min"))
amp_max = float(parser.get("Sampling", "amp_max"))

rad_min = float(parser.get("Sampling", "rad_min"))
rad_max = float(parser.get("Sampling", "rad_max"))

prefix = parser.get("Misc", "prefix")
location = parser.get("Misc", "location")
output_folder = location + "/" + prefix 

x,y,r,a,L = np.loadtxt(output_folder + "/" + prefix + "_out_points_som.txt", unpack=True)

all_vals = np.vstack((x,y,r,a,L))

In [95]:
q = np.array([[1,2,3],[4,5,6]])

In [96]:
%matplotlib inline

In [97]:
def uniqueish_color():
    """There're better ways to generate unique colors, but this isn't awful."""
    return plt.cm.gist_ncar(np.random.random())

In [98]:
bin_amt = 350

In [99]:
def next_dim(dim):
    if dim == 0:
        return 1
    return 0


In [100]:
q = np.array([[1,2],[3,4]])

In [101]:
q

array([[1, 2],
       [3, 4]])

In [102]:
q.flatten()

array([1, 2, 3, 4])

In [103]:
def get_peaks(initial_bounds):
    queue = []
    results = [] #going to store results as (depth, [xmin, xmax, ymin, ymax])
    
    queue.append((0, 0, initial_bounds))#depth, dim, [[xlower, xupper],[ylower, yupper]]
    while queue != []:
        depth, dim, bounds = queue.pop()
        start, stop = bounds[dim]
        other_start, other_stop = bounds[next_dim(dim)]
        
        print "bounds"
        print bounds
        
        if start == stop or other_start == other_stop:
            continue
        
        dimvals = all_vals[:, dim].T
        range_mask = np.where((dimvals >= start) & (dimvals <= stop))
        dimvals = dimvals[range_mask]
        lvals = all_vals[range_mask, dim].T
        
        _, main_mask, main_binned, main_binned_L = common.binned_max(dimvals, lvals, start, stop, bin_amt) 
        
        if main_binned_L.shape[0] == 0:
            #there nothing here
            continue
        main_smoothed = common.smooth(main_binned_L[main_mask])
        
        #check if there is a peak
        median = np.median(main_smoothed)
        peak = np.max(main_smoothed)
        if peak < 0.999 * median:
            continue
        else:
            print "peak %f"%peak
            results.append((depth, bounds.flatten()))
        
        main_mins = common.compute_mins(main_binned[main_mask], main_smoothed)
        main_maxes = common.compute_maxes(main_binned[main_mask], main_smoothed)
        main_intervals = common.compute_intervals(main_mins, main_maxes)
        main_intervals = np.floor(main_intervals).astype("int")
        
        if main_intervals.shape[0] == 0: #no intervals to look at
            print "no intervals to look at"
            print main_smoothed
            print main_mins, main_maxes
            continue
        
        for nstart, nstop in intervals:
        
            other_col = all_vals[:, next_dim(dim)]
            my_col = all_vals[:, dim]
            my_mask = np.where((my_col >= nstart) & (my_col <= nstop))
        
            _, my_mask, my_binned, my_binned_L = common.binned_max(other_col[my_mask], L[my_mask], other_start, other_stop, 50) 
        
        
            if my_binned_L[my_mask].shape[0] == 0:
                print "binning failed"
                continue
        
            my_smoothed = common.smooth(my_binned_L[my_mask])

            my_mins = common.compute_mins(my_binned[my_mask], my_smoothed)
            my_maxes = common.compute_maxes(my_binned[my_mask], my_smoothed)
            my_intervals = common.compute_intervals(my_mins, my_maxes)
            my_intervals = np.floor(intervals).astype("int")
            
            for my_start, my_stop in my_intervals:
                b = np.zeros((2,2))
                b[dim] = nstart, nstop
                b[next_dim(dim)] = my_start, my_stop
                queue.append((depth, next_dim(dim), b))
                print my_start, my_stop
                
    return results

In [104]:
initial_bounds = np.array([[0, width], [0, height]])
k = get_peaks(initial_bounds)

bounds
[[  0 200]
 [  0 200]]
peak 22.235704
no intervals to look at
[ 22.23570357  22.23570357  22.23570357  22.23570357]
[] []


In [105]:
k

[(0, array([  0, 200,   0, 200]))]

In [106]:
def get_peaks(bounds, dim, debug_plots = False):
    start, stop = bounds[dim]
    other_start, other_stop = bounds[next_dim(dim)]
    
    if start == stop:
        return []
    lvals = L
    dimvals = all_vals[dim]
    
    _, mask, binned, binned_L = common.binned_max(dimvals, lvals, start, stop, bin_amt) 
    if binned_L[mask].shape[0] == 0:
        return []
    smoothed = common.smooth(binned_L[mask])
    
    #Check to see if there is even a possible peak
    median = np.median(smoothed)
    peak = np.max(smoothed)
    print peak, median
    if peak < 0.999 * median:
        #some criteria
        return []
    
    index = np.where(smoothed == peak)[0]
    print index
    print "peak is at"
    print all_vals[dim, index]
    
    mins = common.compute_mins(binned[mask], smoothed)
    maxes = common.compute_maxes(binned[mask], smoothed)
    intervals = common.compute_intervals(mins, maxes)
    intervals = np.floor(intervals).astype("int")
    
    if intervals.shape[0] == 0:
        return []
    
    new_bounds = []
    
    print intervals

    for nstart, nstop in intervals:
        
        other_col = all_vals[:, next_dim(dim)].T
        my_col = all_vals[:, dim].T
        my_mask = np.where((my_col >= nstart) & (my_col <= nstop))
        
        _, my_mask, my_binned, my_binned_L = common.binned_max(other_col[my_mask], L[my_mask], other_start, other_stop, 50) 
        
        if my_binned_L[my_mask].shape[0] == 0:
            continue
        
        my_smoothed = common.smooth(my_binned_L[my_mask])

        my_mins = common.compute_mins(my_binned[my_mask], my_smoothed)
        my_maxes = common.compute_maxes(my_binned[my_mask], my_smoothed)
        my_intervals = common.compute_intervals(mins, maxes)
        my_intervals = np.floor(intervals).astype("int")
        
        for my_start, my_stop in my_intervals:
            b = np.zeros((2,2))
            b[dim] = nstart, nstop
            b[next_dim(dim)] = my_start, my_stop
            new_bounds.append(b)
            
    print "new bounds"
    print new_bounds
    return [(dim, [start, stop], get_peaks(b, next_dim(dim), debug_plots=debug_plots)) for b in new_bounds]

In [107]:
pdb on

Automatic pdb calling has been turned ON


In [ ]:
initial_bounds = np.array([[0, width], [0, height]])
k = get_peaks(initial_bounds, 0, debug_plots=False)

In [ ]:
k

Below is still a work in progress

In [ ]:
def post_process(tree, depth=0):
    curr_dim = tree[0]
    dim_bounds = tree[1]
    sub_bounds = tree[2]
    if sub_bounds == []:
        return []
    my_sub_bounds = []
    for sb in sub_bounds:
        if sb[-1] == []:
            my_sub_bounds.append(sb)
    return_vals = [(depth, [dim_bounds, sb[1]]) for sb in my_sub_bounds]
    for sub_tree in sub_bounds:
        return_vals += post_process(sub_tree, depth+1)
    return return_vals

def get_sources(tree):
    res = []
    for subtree in k:
        res += post_process(subtree)
    return res

In [ ]:
def get_sources(tree):
    res = [post_process(subtree) for subtree in k]
    res = filter(lambda x: x != [], res)
    return_val = []
    for part in res:
        entry = max(part, key = lambda item: item[0])
        deepest = entry[0]
        
        #now that we know the deepest, we can go and keep only the last calls
        keep = filter(lambda item: item[0] == deepest, part)
        return_val += keep
    new_rv = []
    for item in return_val:
        t = [item[0]] +  item[1][0] + item[1][1]
        new_rv.append(t)
    return new_rv
        

In [ ]:
out = get_sources(k)

In [ ]:
temp_sources = np.array(out)
#depth, xmin, xmax, ymin, ymax

In [ ]:
temp_sources.shape[0]

In [ ]:
temp_sources

In [ ]:
depth, xlower, xupper, ylower, yupper = temp_sources[0]


In [ ]:
np.where((x >= xlower) & (x <= xupper) & (y >= ylower) & (y <= yupper))[0]

In [ ]:
sources = np.zeros((temp_sources.shape[0], 6))
#x,y,r,a,depth, meanL

for i in xrange(temp_sources.shape[0]):
    depth, xlower, xupper, ylower, yupper = temp_sources[i]
    mask = np.where((x >= xlower) & (x <= xupper) & (y >= ylower) & (y <= yupper))[0]
    
    sources[i, 0] = np.mean(x[mask])
    sources[i, 1] = np.mean(y[mask])
    sources[i, 2] = np.dot(r[mask], L[mask])/np.sum(L[mask])
    sources[i, 3] = np.dot(a[mask], L[mask])/np.sum(L[mask])
    sources[i, 4] = temp_sources[i,0]
    sources[i, 5] = np.mean(L[mask])

In [ ]:
import pandas as pd

In [ ]:
s_frame = pd.DataFrame(sources, columns = ["x", "y", "r", "a", "depth", "meanL"])
s_frame = s_frame.drop_duplicates()
s_frame = s_frame.sort(columns=["meanL"], ascending=False)

In [ ]:
s_frame.shape

In [ ]:
plt.scatter(s_frame.x, s_frame.y)

In [ ]:
np.mean(s_frame.r)